In [ ]:
# Takes two images with the detector at different positions, aligns using central module 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave

from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from scipy.ndimage import fourier_shift, shift

In [2]:
im1 = imread('CBr4_d700_333K_0p0-16977.tif')
im2 = imread('CBr4_d700_333K_2_0p0-20581.tif')
im1[im1 < 0] = 0
im2[im2 < 0] = 0
#im1[42:52,42:52] = 10000000

In [4]:
plt.subplot(1, 2, 1)
plt.imshow(im1, vmax = 500)
plt.subplot(1, 2, 2)
plt.imshow(im2, vmax = 500)
plt.show()

array([-10.,  -5.])

In [7]:
test = shift(im1, (10.5, 5), order = 5)
test[test<0]= 0

In [9]:
plt.subplot(1, 2, 1)
plt.imshow(im1, vmax = 700)
plt.subplot(1, 2, 2)
plt.imshow(test, vmax = 700)
plt.show()

In [75]:
imshift

array([0., 0.])

In [86]:
plt.imshow(im1, vmax = 1000)
plt.show()

In [103]:
plt.imshow(im1[848:1043:,494:737], vmax = 1000)
plt.show()

In [4]:
#imshift, error, diffphase = register_translation(im1[848:1043:,494:737],im2[848:1043:,494:737])
#imshift, error, diffphase = register_translation(im1[848:1043:,494:981],im2[848:1043:,494:981], 10)
# Get center module with a bunch of diffuse features
c1 = im1[839-200:839+200,737-200:737+200]
c2 = im2[839-200:839+200,737-200:737+200]
# Determine shift within 1 pixel
imshift, error, diffphase = register_translation(c1, c2, 1)

In [5]:
print(imshift)

[-84. -67.]


In [11]:
#test = np.copy(im2)
#test[42-int(imshift[0]):52-int(imshift[0]),42-int(imshift[1]):52-int(imshift[1])] = 10000000
#test = shift(test, imshift, order = 3)
#test[test<0]= 0
sim2 = np.copy(im2)
sim2[42-int(imshift[0]):52-int(imshift[0]),42-int(imshift[1]):52-int(imshift[1])] = 10000000# Indexing test

# Define how much to pad edges of images
paddings = ((np.int(np.abs(imshift[0])), np.int(np.abs(imshift[0]))), (np.int(np.abs(imshift[1])), np.int(np.abs(imshift)[1])))
# Pad images so edges aren't thrown away
sim1 = np.pad(im1, paddings, 'constant', constant_values = ((0,0)))
sim2 = np.pad(sim2, paddings, 'constant', constant_values = ((0,0)))
# Shift image
sim2 = shift(sim2, imshift, order = 3)

In [12]:
plt.subplot(1, 2, 1)
plt.imshow(sim1, vmax = 700)
plt.subplot(1, 2, 2)
plt.imshow(sim2, vmax = 700)
plt.show()

In [13]:
plt.imshow(np.add(sim1, sim2), vmax = 700)
plt.show()

In [17]:
# Mask containing nonzero pixels mask
mask1 = np.zeros(np.shape(sim1))
mask1[sim1!=0] = 1
mask2 = shift(mask1, imshift, order = 1) # This may need to be replaced with np.roll
# Normalization mask
norm = np.add(mask1, mask2)

In [21]:
plt.plot(mask2[300])
plt.show()

In [22]:
normed = np.divide(np.add(sim1, sim2), norm)
plt.imshow(normed, vmax = 500)
plt.show()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [348]:
plt.imshow(test)
plt.show()

In [224]:
plt.imshow(mask2, vmax = 1)
plt.show()

In [23]:
# Doesn't work because nans are a thing
#imsave('normed.tiff', np.nan_to_num(normed)
np.save('normed_fixed.npy', normed)

In [264]:
plt.imshow(im1[839-200:839+200,737-200:737+200], vmax = 600)
plt.show()

In [266]:
from skimage.exposure import equalize_adapthist, equalize_hist

In [375]:
plt.imshow(np.log(np.nan_to_num(normed[760:860, 595:695])))
plt.show()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


In [378]:
from skimage.restoration import inpaint

In [419]:
inpaint_mask = np.copy(normed)
inpaint_mask[np.isfinite(inpaint_mask)] = 1
inpaint_mask[np.isnan(inpaint_mask)] = 0
image_defect = np.copy(normed)

In [423]:
plt.imshow(normed, vmax = 700)
plt.show()

In [420]:
plt.imshow(image_defect)
plt.show()

In [424]:
image_result = inpaint.inpaint_biharmonic(image_defect, inpaint_mask)

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


KeyboardInterrupt: 